In [ ]:
#Ashiqur RahmanKhan
#online ApacheSpark installation

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.6.tgz
!tar xvf spark-2.4.5-bin-hadoop2.6.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.6"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

In [3]:
!wget https://github.com/shatiilrahman/Machine-Learning/blob/master/titanic.parquet?raw=true
!mv titanic.parquet?raw=true titanic.parquet

--2020-05-11 21:06:06--  https://github.com/shatiilrahman/Machine-Learning/blob/master/titanic.parquet?raw=true
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/shatiilrahman/Machine-Learning/raw/master/titanic.parquet [following]
--2020-05-11 21:06:06--  https://github.com/shatiilrahman/Machine-Learning/raw/master/titanic.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/shatiilrahman/Machine-Learning/master/titanic.parquet [following]
--2020-05-11 21:06:06--  https://raw.githubusercontent.com/shatiilrahman/Machine-Learning/master/titanic.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import OneVsRest
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
#LogisticRegression with OneVsRest

In [0]:
#loading dataset
d=0
d = spark.read.parquet('/content/titanic.parquet')
d.createOrReplaceTempView("titanic")
(d_train,d_test) = d.randomSplit([0.7, 0.3])

In [18]:
d.show()

+-----------+--------+------+--------------------+------+----+-----+-----+---------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|         Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+---------------+-------+-----+--------+
|          9|     1.0|   3.0|Johnson, Mrs. Osc...|female|27.0|  0.0|  2.0|         347742|11.1333| null|       S|
|         64|     0.0|   3.0|Skoog, Master. Ha...|  male| 4.0|  3.0|  2.0|         347088|   27.9| null|       S|
|        168|     0.0|   3.0|Skoog, Mrs. Willi...|female|45.0|  1.0|  4.0|         347088|   27.9| null|       S|
|        228|     0.0|   3.0|"Lovell, Mr. John...|  male|20.5|  0.0|  0.0|      A/5 21173|   7.25| null|       S|
|        322|     0.0|   3.0|    Danoff, Mr. Yoto|  male|27.0|  0.0|  0.0|         349219| 7.8958| null|       S|
|        440|     0.0|   2.0|Kvillner, Mr. Joh...|  male|31.0|  0.0|  0.0|     C.A. 1872

In [0]:
#pipeline formation
indexer = StringIndexer(inputCol = "Sex" , outputCol = "label")
vectorAssembler = VectorAssembler(inputCols=["Survived","Pclass","SibSp","Parch","Fare"],outputCol="features")
normalizer = Normalizer(inputCol="features", outputCol="features_norm",p=1.0)
classifier = LogisticRegression(maxIter = 20, regParam = 0.3, elasticNetParam = 0.8)
ovr = OneVsRest(classifier=classifier)
pipelineLo = Pipeline(stages=[indexer, vectorAssembler, normalizer,ovr])

In [9]:
#model_fit
modelLo = pipelineLo.fit(d_train)
prediction = modelLo.transform(d_test)
eval = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
eval.evaluate(prediction) 
accuracyLo = eval.evaluate(prediction)
print("LogisticRegressionOneVsRest Accuracy : ",accuracyLo)

LogisticRegressionOneVsRest Accuracy :  0.6752767527675276


In [0]:
#RandomForestClassifier

In [0]:
#loading dataset
df_temp = prediction
data = df_temp.drop("prediction")
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [17]:
data.show()

+-----------+--------+------+--------------------+------+----+-----+-----+---------------+-------+-----+--------+-----+--------------------+--------------------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|         Ticket|   Fare|Cabin|Embarked|label|            features|       features_norm|
+-----------+--------+------+--------------------+------+----+-----+-----+---------------+-------+-----+--------+-----+--------------------+--------------------+
|        106|     0.0|   3.0|Mionoff, Mr. Stoy...|  male|28.0|  0.0|  0.0|         349207| 7.8958| null|       S|  0.0|(5,[1,4],[3.0,7.8...|(5,[1,4],[0.27533...|
|        108|     1.0|   3.0|Moss, Mr. Albert ...|  male|null|  0.0|  0.0|         312991|  7.775| null|       S|  0.0|[1.0,3.0,0.0,0.0,...|[0.08492569002123...|
|        125|     0.0|   1.0|White, Mr. Perciv...|  male|54.0|  0.0|  1.0|          35281|77.2875|  D26|       S|  0.0|[0.0,1.0,0.0,1.0,...|[0.0,0.0126123285...|
|        133|     0.0|   3.0

In [0]:
#pipeline formation
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)
pipelineRa = Pipeline(stages=[labelIndexer, featureIndexer, rf])

In [0]:
#model_fit
modelRa = pipelineRa.fit(trainingData)
prediction = modelRa.transform(testData)

In [13]:
evalua = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
evalua.evaluate(prediction)
accuracyRa = evalua.evaluate(prediction)
print("RandomForestClassifier Accuracy : ",accuracyRa)

RandomForestClassifier Accuracy :  0.7204301075268817
